## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,57.85,77,0,5.03,clear sky
1,1,Isangel,VU,-19.5500,169.2667,81.57,67,10,1.19,clear sky
2,2,Avarua,CK,-21.2078,-159.7750,78.67,74,58,2.71,broken clouds
3,3,Cabo San Lucas,MX,22.8909,-109.9124,81.61,53,0,4.61,clear sky
4,4,Nikolskoye,RU,59.7035,30.7861,40.91,88,100,5.41,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Isangel,VU,-19.5500,169.2667,81.57,67,10,1.19,clear sky
2,2,Avarua,CK,-21.2078,-159.7750,78.67,74,58,2.71,broken clouds
3,3,Cabo San Lucas,MX,22.8909,-109.9124,81.61,53,0,4.61,clear sky
8,8,Kapaa,US,22.0752,-159.3190,80.58,67,0,14.97,clear sky
10,10,Hambantota,LK,6.1241,81.1185,80.02,82,36,13.58,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
664,664,Halayhayin,PH,14.1891,121.4075,87.40,62,100,2.17,overcast clouds
669,669,Lahaina,US,20.8783,-156.6825,83.44,73,8,9.78,clear sky
672,672,Palauig,PH,15.4350,119.9045,86.79,68,67,8.23,broken clouds
675,675,Puerto Gaitan,CO,4.3133,-72.0816,77.59,85,89,4.36,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                211
City                   211
Country                211
Lat                    211
Lng                    211
Max Temp               211
Humidity               211
Cloudiness             211
Wind Speed             211
Current Description    211
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Isangel,VU,81.57,clear sky,-19.5500,169.2667,
2,Avarua,CK,78.67,broken clouds,-21.2078,-159.7750,
3,Cabo San Lucas,MX,81.61,clear sky,22.8909,-109.9124,
8,Kapaa,US,80.58,clear sky,22.0752,-159.3190,
10,Hambantota,LK,80.02,scattered clouds,6.1241,81.1185,
13,Vaini,TO,82.56,few clouds,-21.2000,-175.2000,
19,Labuhan,ID,80.40,overcast clouds,-6.8844,112.2051,
28,Hilo,US,88.29,broken clouds,19.7297,-155.0900,
30,Minapasuk,PH,82.45,overcast clouds,10.6988,123.3536,
39,Awjilah,LY,82.42,clear sky,29.1081,21.2869,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:  
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df[hotel_df["Hotel Name"] == ""]
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Isangel,VU,81.57,clear sky,-19.5500,169.2667,Tanna Lodge
2,Avarua,CK,78.67,broken clouds,-21.2078,-159.7750,Paradise Inn
3,Cabo San Lucas,MX,81.61,clear sky,22.8909,-109.9124,Vista Roof Restaurant
8,Kapaa,US,80.58,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
10,Hambantota,LK,80.02,scattered clouds,6.1241,81.1185,Bungalow 63


In [21]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search_csv/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather </dt><dd> {Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))